In [2]:
!pip install pytorch-lightning --quiet

     |████████████████████████████████| 701 kB 8.6 MB/s 
     |████████████████████████████████| 141 kB 49.8 MB/s 
     |████████████████████████████████| 596 kB 44.3 MB/s 
     |████████████████████████████████| 5.9 MB 32.0 MB/s 
     |████████████████████████████████| 419 kB 53.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


In [3]:
import os
import pandas as pd
import numpy as np
import cv2
import torch
from torch import nn
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset ,DataLoader
from torchvision.datasets.utils import download_url
from pytorch_lightning import LightningDataModule, callbacks, Trainer, LightningModule
from sklearn.model_selection import train_test_split
import requests
import matplotlib.pyplot as plt
import torchmetrics as tm
from pytorch_lightning.loggers import TensorBoardLogger

In [4]:
if not os.path.exists('food-101.tar.gz'):
    print('Downloading Food101 dataset!')
    download_url('http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz', '.')

  0%|          | 0/4996278331 [00:00<?, ?it/s]

In [5]:
# Unzip the files downloaded
!tar -xf food-101.tar.gz

In [6]:
# config
images_path = './food-101/images'
img_size = 512

In [7]:
# config
images_path = './food-101/images'
img_size = 512

In [8]:
# sample 5 class names to check if the dataset is correct
list_classes = os.listdir(images_path)
dict_classes = dict(zip(list_classes, range(0,len(list_classes))))
dict_class_ids = {v: k for k, v in dict_classes.items()}
n_classes = len(list_classes)
list_classes[:5]

['chicken_quesadilla',
 'breakfast_burrito',
 'filet_mignon',
 'prime_rib',
 'macaroni_and_cheese']

In [9]:
# create a csv of all the image files available in the dataset
files_df = pd.DataFrame(columns=['file_names', 'label'])
for root, dirs, files in os.walk(images_path, topdown=False):
    for class_name in dirs:
        list_files = os.listdir(os.path.join(root, class_name))
        files = pd.DataFrame({'file_names': list_files, 'label': [class_name]*len(list_files),
                              'label_id': [int(dict_classes[class_name])]*len(list_files)})
        files_df = pd.concat([files_df, files])
files_df.to_csv(os.path.join(images_path, 'files.csv'), index=False)

In [10]:
# create a torch dataset
class FoodDataset(Dataset):
    
    def __init__(self, base_path, file_names, labels, image_size):
        self.file_names = file_names
        self.labels = labels
        self.base_path = base_path
        self.img_size = img_size
    
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, item):
        file_name = str(self.file_names[item])
        label = self.labels[item]
        img_file = cv2.imread(os.path.join(self.base_path, dict_class_ids[np.int64(label)], file_name))
        if self.img_size is not None:
            img = cv2.resize(img_file,(self.img_size, self.img_size))
        img = img.astype(np.float64)
        return {"x":torch.tensor(img, dtype=torch.float), "y":torch.tensor(np.int64(label), dtype=torch.long)}

In [11]:
# Create a pytorch lightning dataset
class FoodLightDataset(LightningDataModule):
    
    def __init__(self, batch_size=128):
        super().__init__()
        self.batch_size = batch_size
    
    def setup(self, stage=None):
        train_df = pd.read_csv(os.path.join(images_path, 'files.csv'))
        
        x_train, x_test, y_train, y_test = train_test_split(train_df.file_names.values, train_df.label_id.values,
                                                            test_size = 0.25, stratify=train_df.label)
        self.train_dataset = FoodDataset(images_path, x_train, y_train, img_size)
        self.validation_dataset = FoodDataset(images_path, x_test, y_test, img_size)
    
    def train_dataloader(self):
        train_loader = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
        return train_loader
    
    def val_dataloader(self):
        valid_loader = DataLoader(self.validation_dataset, batch_size=self.batch_size, shuffle=False)
        return valid_loader

In [12]:
train_df = pd.read_csv(os.path.join(images_path, 'files.csv'))
x_train, x_test, y_train, y_test = train_test_split(train_df.file_names.values, train_df.label_id.values,
                                                    test_size = 0.2, stratify=train_df.label)
train_dataset = FoodDataset(images_path, x_train, y_train, img_size)
validation_dataset = FoodDataset(images_path, x_test, y_test, img_size)
fld = FoodLightDataset()
fld.setup()

In [13]:
len(fld.train_dataset.labels)

75750

In [14]:
len(fld.validation_dataset.labels)

25250

# A Basic CNN

In [15]:
from torchvision import models

class TransferLearningModel(LightningModule):
    
    def __init__(self):
        super().__init__()
        
        backbone = models.resnet50(pretrained=True)
        for param in backbone.parameters():
            param.requires_grad = False
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)

        # use the pretrained model to classify food101
        self.classifier = nn.Linear(num_filters, n_classes)

        self.flat = nn.Flatten()

        self.accuracy = tm.functional.accuracy

    def forward(self,x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)
        
        return x

    def loss_fn(self,out,target):
        return nn.CrossEntropyLoss()(out.view(-1, n_classes), target)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(),lr=1e-3)
        return optimizer

    def training_step(self,batch,batch_idx):
        inp, target = batch['x'], batch['y']
        target = target.view(-1)
        output = self(inp.view(-1,3,img_size,img_size))
        loss = self.loss_fn(output, target)
        self.log("train_loss", loss)
        return loss

    def validation_step(self,batch,batch_idx):
        inp, target = batch['x'], batch['y']
        target = target.view(-1)
        output = self(inp.view(-1,3,img_size,img_size))
        loss = self.loss_fn(output, target)
        self.log("val_loss", loss)
    
    def test_step(self, batch, batch_idx):
        x, y = batch["x"], batch["y"]
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        prec = self.accuracy(y_hat, y)
        self.log("test_accuracy", prec[0])
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
checkpoint_callback = callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath='./',
    filename='models-{epoch:02d}-{valid_loss:.2f}',
    save_top_k=3,
    mode='min')

logger = TensorBoardLogger('transfer_learning_logs', name='Transfer learning CNN Model')
mod = TransferLearningModel()
dx = FoodLightDataset()
trainer = Trainer(max_epochs=3, callbacks=[checkpoint_callback], 
                  logger=logger)
trainer.fit(model=mod, datamodule=dx)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1769: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /content exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type       | Params
-------------------------------------------------
0 | feature_extractor | Sequential | 23.5 M
1 | classi

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
mod = BasicCNNModel()

In [ ]:
res = next(iter(fld.train_dataloader()))
res['x'].shape
res['x'].view(-1, 3, img_size, img_size).shape
# mod(res['x'].view(-1,3,img_size, img_size))

torch.Size([32, 3, 128, 128])

In [ ]:
features = mod.features(res['x'].view(-1,3,img_size, img_size))

In [ ]:
res = mod(res['x'].view(-1,3,img_size,img_size))

In [ ]:
logits = torch.argmax(res,dim=0)

In [ ]:
torch.unique(res)

tensor([1.4013e-45, 4.2039e-45, 9.8091e-45,  ..., 1.0000e+00, 1.0000e+00,
        1.0000e+00], grad_fn=<Unique2Backward0>)

In [ ]:
len(logits)

101

In [ ]:
torch.unique(logits)

tensor([ 0,  1,  2,  5,  8, 11, 13, 15, 22, 24, 26, 27])

In [ ]:
len(logits)

101

In [ ]:
accu = mod.accuracy(logits, torch.from_numpy(np.array([86 for x in range(len(logits))], dtype=np.int64)))

In [ ]:
res.shape

torch.Size([32, 101])

In [ ]:
checkpoint_callback = callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath='./',
    filename='models-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min')

logger = TensorBoardLogger('cnn_logs', name='Basic CNN Model')

mod = BasicCNNModel()
dx = FoodLightDataset()
trainer = Trainer(gpus=-1, max_epochs=30, callbacks=[checkpoint_callback], 
                  accelerator='gpu', logger=logger)
trainer.fit(model=mod, datamodule=dx)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /content exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
IN

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir cnn_logs/